In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def process_transaction(transaction_ref):
    # Split string using /
    parts = str(transaction_ref).split("/")
    
    # Extract Payment Type (first token, lowercase)
    payment_type = parts[0].lower()
    
    # Extract Merchant (index 3, lowercase, stripped)
    merchant = None
    if len(parts) > 3:
        merchant = parts[3].lower().strip()
    
    # Categorize Merchant using keyword matching
    category = "others"
    if merchant:
        if any(kw in merchant for kw in ['swiggy', 'zomato', 'restaurant', 'pizza', 'kfc', 'mcdonalds']):
            category = "food"
        elif any(kw in merchant for kw in ['netflix', 'spotify', 'prime', 'hotstar', 'multiplex', 'cinema']):
            category = "entertainment"
        elif any(kw in merchant for kw in ['udemy', 'coursera', 'byjus', 'edx', 'school', 'college']):
            category = "education"
        elif any(kw in merchant for kw in ['uber', 'ola', 'rapido', 'irctc', 'indigo', 'airindia', 'railway']):
            category = "travel"
        elif any(kw in merchant for kw in ['amazon', 'flipkart', 'myntra', 'dmart', 'bigbazaar', 'ajio', 'reliance']):
            category = "shopping"
        elif any(kw in merchant for kw in ['electricity', 'water', 'gas', 'bill', 'bescom', 'recharge']):
            category = "utilities"
        elif any(kw in merchant for kw in ['salary', 'payroll', 'stipend']):
            category = "salary"
        elif any(kw in merchant for kw in ['apollo', 'pharmeasy', 'hospital', 'medical', 'pharmacy', 'health']):
            category = "medical"
        elif any(kw in merchant for kw in ['jio', 'vi', 'airtel', 'bsnl', 'telecom']):
            category = "recharge"
            
    return payment_type, category

# 1. Load the dataset
df = pd.read_csv('./data/RawDataset.csv')

# 2. Add payment_type and category columns
source_col = 'Description' if 'Description' in df.columns else 'Transaction Reference'
df[['payment_type', 'category']] = df.apply(lambda x: process_transaction(x[source_col]), axis=1, result_type='expand')

# 3. Convert Value Date to datetime and extract features
df['Value Date'] = pd.to_datetime(df['Value Date'], dayfirst=True)
df['day'] = df['Value Date'].dt.day
df['month'] = df['Value Date'].dt.month
df['year'] = df['Value Date'].dt.year

# 4. Remove specified columns
# Columns to remove: Txn Date, Description (source_col), Balance
cols_to_drop = ['Txn Date', source_col, 'Balance']
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# 5. Convert payment_type and category to numeric
le_payment = LabelEncoder()
le_category = LabelEncoder()

df['payment_type'] = le_payment.fit_transform(df['payment_type'])
df['category'] = le_category.fit_transform(df['category'])

# Fill remaining NA values with 0
df.fillna(0, inplace=True)

# Show the first few rows of the processed dataset
print(df.head())


,Txn Date,Value Date,Description,Ref No./Cheque No.,Debit,Credit,Balance,payment_type,category
0,01/01/2022,01/01/2022,NEFT/CR/952735718030/SALARY,TRANSFER FROM 952735718030,0.00,125213.0,431114.00,neft,salary
1,01/01/2022,01/01/2022,POS/DR/711995879155/MYNTRA,TRANSFER TO 711995879155,1665.43,0.0,429448.57,pos,shopping
2,01/01/2022,01/01/2022,IMPS/DR/821490367535/BIGBAZAAR,TRANSFER TO 821490367535,1563.68,0.0,427884.89,imps,others
3,01/01/2022,01/01/2022,NEFT/DR/183967131867/FLIPKART,TRANSFER TO 183967131867,3419.58,0.0,424465.31,neft,shopping
4,01/01/2022,01/01/2022,UPI/DR/858057510226/FLIPKART,TRANSFER TO 858057510226,6077.98,0.0,418387.33,upi,shopping
...,...,...,...,...,...,...,...,...,...
29573,12/09/2024,12/09/2024,UPI/DR/628741509522/APOLLO,TRANSFER TO 628741509522,50.00,0.0,-14270921.33,upi,others
29574,12/09/2024,12/09/2024,IMPS/DR/937576857874/JIO,TRANSFER TO 937576857874,65.71,0.0,-14270987.04,imps,others
29575,12/09/2024,12/09/2024,POS/DR/340652065299/BIGBAZAAR,TRANSFER TO 340652065299,50.00,0.0,-14271037.04,pos,others
29576,12/09/2024,12/09/2024,IMPS/DR/124419417383/OLA,TRANSFER TO 124419417383,112.57,0.0,-14271149.61,imps,transport
